In [1]:
import logging

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")

# Create a console handler and set its level
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)

# Create a formatter and set it on the console handler
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
console_handler.setFormatter(formatter)

# Add the console handler to the logger
logger.addHandler(console_handler)


def main():
    logger.info("Cloud function starting.")
    # 1. Scrape coingecko gainers.
    import scraping.scraper

    scraping.scraper.main()

    # 2. Get contract address for gainers that have them, with coingecko.
    import scraping.contract_addresses

    scraping.contract_addresses.main()

    # 3. Run SQL query on Google Big Query to find wallets that have receieved these tokens within the last 7 days.


main()


2023-04-03 16:21:45,461 - INFO - Cloud function starting.
2023-04-03 16:21:46,704 - INFO - Scraping complete.
2023-04-03 16:21:47,511 - INFO - Scraping 3 coins that are missing symbols.
2023-04-03 16:21:51,028 - INFO - Scraped https://www.coingecko.com/en/coins/sxp
2023-04-03 16:21:54,962 - INFO - Scraped https://www.coingecko.com/en/coins/injective
2023-04-03 16:21:58,572 - INFO - Scraped https://www.coingecko.com/en/coins/panther-protocol
2023-04-03 16:21:58,586 - INFO - Number of coins with ethereum contract addresses: 19
2023-04-03 16:21:58,587 - INFO - Done scraping contract addresses!


In [2]:
import pandas as pd

with open("gainers.csv") as f:
    df = pd.read_csv(f)

token_contracts = [f"'{contract}'" for contract in df["contract_address"]]
token_contracts_string = f"({','.join(token_contracts)})"
logger.info(f"Checking {len(token_contracts)} token contracts.")


import os

# Load environment variables from .env file
from dotenv import load_dotenv

load_dotenv()
KEY_PATH = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
PROJECT_ID = os.getenv("PROJECT_ID")

from google.cloud import bigquery
from google.oauth2 import service_account

# Authenticate to BigQuery
credentials = service_account.Credentials.from_service_account_file(
    KEY_PATH,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)
client = bigquery.Client(
    credentials=credentials,
    project=credentials.project_id,
)

dry_run_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
wet_run_config = bigquery.QueryJobConfig(use_query_cache=False)

# Query the last 7 days of token transfers
query = f"""
WITH wallet_token_counts AS (
  SELECT tt.to_address, tt.token_address, COUNT(tt.to_address) as num_txns
  FROM `bigquery-public-data.crypto_ethereum.token_transfers` tt 
  WHERE tt.block_timestamp > TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 7 DAY) 
  AND tt.block_timestamp < TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 2 DAY) 
  AND tt.token_address in {token_contracts_string}
  GROUP BY tt.to_address, tt.token_address)

SELECT to_address as wallet, COUNT(to_address) as num_tokens, SUM(num_txns) as num_txns
FROM wallet_token_counts wtc 
LEFT JOIN `bigquery-public-data.crypto_ethereum.contracts` c
ON wtc.to_address = c.address
LEFT JOIN `labels.labels` labels
ON wtc.to_address = labels.address
WHERE c.address IS NULL
AND labels.address IS NULL
GROUP BY to_address
HAVING num_tokens >= 3
ORDER BY num_txns ASC, num_tokens DESC
"""

logger.info("Running SQL query on Google Big Cloud.")
dry_query = client.query(query, job_config=dry_run_config)

# Check how much data will be processed
mb_processed = dry_query.total_bytes_processed / (1024 * 1024 * 4)
# Throw error if over 1024*4 MB
if mb_processed > 1024:
    logger.error(f"Query will process {mb_processed:.2f} MB.")
    raise Exception(f"Query will process {mb_processed:.2f} MB.")

logger.info(f"Query will process {mb_processed:.0f} MB.")

# Run query
query_job = client.query(query, job_config=wet_run_config)


2023-04-03 16:22:01,730 - INFO - Checking 19 token contracts.
2023-04-03 16:22:01,890 - INFO - Running SQL query on Google Big Cloud.
2023-04-03 16:22:05,358 - INFO - Query will process 768 MB.


In [3]:
# Turn query job into df
df = query_job.to_dataframe()

In [4]:
df

,wallet,num_tokens,num_txns
0,0xb758b6576221a7504a7211307092c23d3ee191c9,3,3
1,0x21375e8ff94a9acad555e0d8a4f17c69c5e33bd5,3,3
2,0x6c1c420c04f4d563d6588a97693af902b87be5f1,3,3
3,0x2511a191af6a389337ca1a3cb692dcb78799b195,3,3
4,0x5828eff23f92e582ddf4fbbc2900e7448e45bd6c,3,3
5,0xcd5bd47d3d1d8412b241ce9015c5032142948c12,3,4
6,0xa683f7bd2a8b36e8799969bf1682f548dbfe8eb6,3,4
7,0x5e8304b5600cccba6d6292c6687ac9ead0ec6288,4,5
8,0x1d283807630ffb876a5d78b8e0788e491449f241,3,5
9,0x9797021d4acc56b30a77d82306d7c3f22aabb5c8,3,5
